# Let's get to training!

## If you have not generated the training data, either run the code from the command line, or go to simulate_interactive.ipynb for  interactive data generation. This notebook details the training of the CNN. Let us start with the imports


In [1]:
import tensorflow as tf
from Nets.Training import *
from Core.LoadTrainingData import DataLoader
from Nets.CNN1D import *

## First, we need to load our training and validation dataset, which we simulated in simulate_interactive.ipynb. Provide the path to both:


In [2]:
training_data   = r'Data\simulated\Train'
validation_data = r'Data\simulated\Val'

## We load the data as follows:

In [3]:
Train_data_loader = DataLoader(training_data)
Val_data_loader = DataLoader(validation_data)

X_Data, Y_Data = Train_data_loader.load_data()
X_Data_val, Y_Data_val = Val_data_loader.load_data()

## Now that the data is loaded, we can instantiate the model. First we define the save path for model weights and architecture

In [4]:
save_path = r'Models/test_train'

## Now, we instantiate the model and save the architecture to json

In [5]:
model =CNN1D((Train_data_loader.params['FragmentSize'],1))
model.summary()


json_config = model.to_json()
with open(os.path.join(save_path,'model-Architecture.json' ), "w") as json_file:
	json_file.write(json_config)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 316, 1)]          0         
                                                                 
 conv1d (Conv1D)             (None, 316, 32)           128       
                                                                 
 batch_normalization (BatchN  (None, 316, 32)          128       
 ormalization)                                                   
                                                                 
 conv1d_1 (Conv1D)           (None, 316, 32)           3104      
                                                                 
 batch_normalization_1 (Batc  (None, 316, 32)          128       
 hNormalization)                                                 
                                                                 
 average_pooling1d (AverageP  (None, 158, 32)          0     

## We can now define the callbacks and compile the model

In [6]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
training_loop = TrainModel(model)
training_loop.compile(optimizer=opt)


save_metriccallb = SaveMetrics(save_path)
save_modelcallb  = SaveBestModel(save_path)

## With everything ready, it is time to fit our model! The training is run for 20 epochs, and model weights at lowest validation loss throughout the training are stored:

In [7]:
training_loop.fit((X_Data,Y_Data),validation_data = (X_Data_val,Y_Data_val), batch_size = 128, epochs=20, verbose = 1,callbacks = [save_metriccallb,save_modelcallb])


Epoch 1/20
182/182 [==============================] - 36s 138ms/step - loss: 0.6711 - val_loss: 0.6466
Epoch 2/20
182/182 [==============================] - 23s 126ms/step - loss: 0.6496 - val_loss: 0.6337
Epoch 3/20
182/182 [==============================] - 23s 126ms/step - loss: 0.6432 - val_loss: 0.6331
Epoch 4/20
182/182 [==============================] - ETA: 0s - loss: 0.6341